In [1]:
import torch
import torchvision
import torchvision.models as models

vgg16 = models.vgg16(pretrained=True)

/Users/AlexHome/vms/effdl-inference/07-software-inference/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Tracing & Scripting

In [3]:
def polino(x):
    return x**2 + 1

In [4]:
def abs_value(x):
    if x.max() >= 0:
        return x
    else:
        return -x

In [5]:
abs_value(torch.tensor(1)), abs_value(torch.tensor(-1))

(tensor(1), tensor(1))

In [6]:
polino(torch.tensor(3))

tensor(10)

In [7]:
abs_traced = torch.jit.trace(abs_value, torch.tensor(-1))

/var/folders/kn/tl5j9yqn33zfgmzx9gxr8mtc0000gq/T/ipykernel_43849/2282889346.py:2: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.max() >= 0:


In [8]:
polino_traced = torch.jit.trace(polino, torch.tensor(0))

In [9]:
polino_traced

In [11]:
polino_traced(torch.tensor(3))

tensor(10)

In [10]:
torch.tensor(3).item()

3

In [12]:
print(abs_traced(torch.tensor(1)), abs_traced(torch.tensor(-1)))

tensor(-1) tensor(1)


In [14]:
def gather(x):
    result = []
    for i in range(3):
        result.append(1)
    return torch.tensor(result)

In [15]:
gather(torch.tensor(6))

tensor([1, 1, 1])

In [16]:
torch.jit.trace(gather, torch.tensor(2))

/var/folders/kn/tl5j9yqn33zfgmzx9gxr8mtc0000gq/T/ipykernel_43849/2080233534.py:5: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return torch.tensor(result)


In [17]:
gather_sc = torch.jit.script(gather)

RuntimeError: 

aten::append.t(t[](a!) self, t(c -> *) el) -> (t[](a!)):
Could not match type int to t in argument 'el': Type variable 't' previously matched to type Tensor is matched to type int.
:
  File "/var/folders/kn/tl5j9yqn33zfgmzx9gxr8mtc0000gq/T/ipykernel_43849/2080233534.py", line 4
    result = []
    for i in range(3):
        result.append(1)
        ~~~~~~~~~~~~~ <--- HERE
    return torch.tensor(result)


In [18]:
abs_script = torch.jit.script(abs_value)

In [19]:
print(abs_script(torch.tensor(1)), abs_script(torch.tensor(-1)))

tensor(1) tensor(1)


In [20]:
def gather(x):
    result: List[int] = []
    for i in range(3):
        result.append(1)
    return torch.tensor(result)

In [22]:
gather_sc = torch.jit.script(gather)

In [23]:
gather_sc(torch.tensor(2))

tensor([1, 1, 1])

In [24]:
example = torch.rand(1, 3, 224, 224)
traced_script_module = torch.jit.trace(vgg16, example)
traced_script_module.save("vgg16.pt")

/Users/AlexHome/vms/effdl-inference/07-software-inference/venv/lib/python3.9/site-packages/torch/jit/_trace.py:983: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1000 / 1000 (100.0%)
Greatest absolute difference: 2.904168963432312 at index (0, 528) (up to 1e-05 allowed)
Greatest relative difference: 657.130832387617 at index (0, 146) (up to 1e-05 allowed)
  _check_trace(


In [25]:
traced_script_module

VGG(
  original_name=VGG
  (features): Sequential(
    original_name=Sequential
    (0): Conv2d(original_name=Conv2d)
    (1): ReLU(original_name=ReLU)
    (2): Conv2d(original_name=Conv2d)
    (3): ReLU(original_name=ReLU)
    (4): MaxPool2d(original_name=MaxPool2d)
    (5): Conv2d(original_name=Conv2d)
    (6): ReLU(original_name=ReLU)
    (7): Conv2d(original_name=Conv2d)
    (8): ReLU(original_name=ReLU)
    (9): MaxPool2d(original_name=MaxPool2d)
    (10): Conv2d(original_name=Conv2d)
    (11): ReLU(original_name=ReLU)
    (12): Conv2d(original_name=Conv2d)
    (13): ReLU(original_name=ReLU)
    (14): Conv2d(original_name=Conv2d)
    (15): ReLU(original_name=ReLU)
    (16): MaxPool2d(original_name=MaxPool2d)
    (17): Conv2d(original_name=Conv2d)
    (18): ReLU(original_name=ReLU)
    (19): Conv2d(original_name=Conv2d)
    (20): ReLU(original_name=ReLU)
    (21): Conv2d(original_name=Conv2d)
    (22): ReLU(original_name=ReLU)
    (23): MaxPool2d(original_name=MaxPool2d)
    (24): 